# Image Impeccable Challenge: Journey to Clarity

## Part 3 : Prediction of Test Data set for Final Submission

By: Leo Dinendra

---

The final submission result file will be saved in the **submission_path** directory, which by default is called the *submission_files* folder.

### 0. General Setting

Put the location of holdout test dataset folder below.

In [1]:
test_data_path = './test_data/'                              # folder where the npy test dataset located

In [2]:
checkpoint_path = './checkpoint/unet_checkpoint_best.pth'  # training checkpoint used

In [3]:
predictions_path = './predictions/'                          # temporary prediction files path

In [4]:
submission_path = './submission_files'                       # final submission format file path

Loading Packages needed.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import pprint as pp
from PIL import Image
from tqdm import tqdm
from skimage.metrics import structural_similarity as sk_ssim
from utils import *
import cv2

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp
from torchmetrics import StructuralSimilarityIndexMeasure

C:\Users\Leo\miniconda3\envs\test_sub\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
if not os.path.exists(predictions_path):
    os.makedirs(predictions_path)

In [8]:
if not os.path.exists(submission_path):
    os.makedirs(submission_path)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
pred_files = []
for root, dirs, files in os.walk(test_data_path):
    for file in files:
        if file.lower().endswith('.npy'):
            full_path = os.path.join(root, file)
            pred_files.append(full_path)

In [11]:
print('there are', len(pred_files), 'files.')

there are 15 files.


### Loading Model & Checkpoint

In [12]:
model = smp.Unet(
    encoder_name="resnet34",        # ResNet as the encoder backbone
    encoder_weights=None,           # start training from scratch for denoising
    in_channels=1,                  
    classes=1,                      
    encoder_depth=4,
    decoder_channels=(128, 64, 32, 16),  # adjusted best decoder channel for this case
    decoder_use_batchnorm=True,     
    decoder_attention_type='scse'     # adding attention in decoder
)
model.segmentation_head = nn.Sequential(
     nn.Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
     nn.ReLU() 
)
model.to(device)
print()

In [13]:
model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

### Prediction

In [14]:
threshold = 0.008 # threshold for predicting only part after water column

In [15]:
for i_file in range(len(pred_files) ):
    print('Predicting', i_file+1, 'of', len(pred_files) )
    pred_noise = np.load(pred_files[i_file], allow_pickle=True, mmap_mode="r+")
    
    # check if the array already has the correct shape
    target_shape = (1259, 300, 300)
    if pred_noise.shape != target_shape:
        pred_noise = np.transpose(pred_noise, axes=(2,1,0))
    
    #placeholder prediction cube for stacking
    pred_clean = np.zeros_like(pred_noise)
    pred_clean2 = np.zeros_like(pred_noise)
    pred_clean3 = np.zeros_like(pred_noise)
    pred_clean4 = np.zeros_like(pred_noise)
    
    model.eval()
    with torch.no_grad():
        for i in tqdm(range(300)):
            seis_noise = pred_noise[:,:,i]
            
            #preprocess
            noise_min = np.min(seis_noise)
            noise_max = np.max(seis_noise)
            seis_noise = (seis_noise - noise_min) / (noise_max - noise_min)
            mean_noise = np.mean(seis_noise)
            seis_noise = seis_noise - mean_noise + 0.5
            diffnoise = np.diff(seis_noise, axis=0)
            seis_noise_ori = seis_noise.copy()
            seis_noise = cv2.resize(seis_noise, (320, 1280))
            data = torch.tensor(seis_noise, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
            
            #prediction
            outputs = model(data)
            output_np = outputs[0, 0, :, :].cpu().numpy()
            
            #postprocess
            output_np = cv2.resize(output_np, (300, 1259))
            output_np_ori= output_np.copy()
            scale = np.max(seis_noise_ori - 0.5, axis=0)/np.max(output_np_ori - 0.5, axis=0)
            seis_noise_ori = seis_noise_ori - 0.5
            seis_noise_ori = seis_noise_ori/scale
            seis_noise_ori = seis_noise_ori + 0.5
            for j in range(300):
                thres = np.where(np.abs(diffnoise[:,j]) > threshold)[0][0]
                output_np[:thres, j] = seis_noise_ori[:thres, j]
            output_np = output_np - 0.5 + mean_noise 
            output_np = output_np * (noise_max - noise_min) + noise_min
            pred_clean[:,:,i] = output_np
            

        for i in tqdm(range(300)):
            seis_noise = pred_noise[:,i,:]
            
            #preprocess
            noise_min = np.min(seis_noise)
            noise_max = np.max(seis_noise)
            seis_noise = (seis_noise - noise_min) / (noise_max - noise_min)
            mean_noise = np.mean(seis_noise)
            seis_noise = seis_noise - mean_noise + 0.5
            diffnoise = np.diff(seis_noise, axis=0)
            seis_noise_ori = seis_noise.copy()
            seis_noise = cv2.resize(seis_noise, (320, 1280))
            data = torch.tensor(seis_noise, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
            
            #prediction
            outputs = model(data)
            output_np = outputs[0, 0, :, :].cpu().numpy()
            
            #postprocess
            output_np = cv2.resize(output_np, (300, 1259))
            output_np_ori= output_np.copy()
            scale = np.max(seis_noise_ori - 0.5, axis=0)/np.max(output_np_ori - 0.5, axis=0)
            seis_noise_ori = seis_noise_ori - 0.5
            seis_noise_ori = seis_noise_ori/scale
            seis_noise_ori = seis_noise_ori + 0.5
            for j in range(300):
                thres = np.where(np.abs(diffnoise[:,j]) > threshold)[0][0]
                output_np[:thres, j] = seis_noise_ori[:thres, j]
            output_np = output_np - 0.5 + mean_noise 
            output_np = output_np * (noise_max - noise_min) + noise_min
            
            pred_clean2[:,i,:] = output_np 
            
        for i in tqdm(range(300)):
            seis_noise = pred_noise[:,:,i]
            seis_noise = np.fliplr(seis_noise)
            
            #preprocess
            noise_min = np.min(seis_noise)
            noise_max = np.max(seis_noise)
            seis_noise = (seis_noise - noise_min) / (noise_max - noise_min)
            mean_noise = np.mean(seis_noise)
            seis_noise = seis_noise - mean_noise + 0.5
            diffnoise = np.diff(seis_noise, axis=0)
            seis_noise_ori = seis_noise.copy()
            seis_noise = cv2.resize(seis_noise, (320, 1280))
            data = torch.tensor(seis_noise, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
            
            #prediction
            outputs = model(data)
            output_np = outputs[0, 0, :, :].cpu().numpy()
            
            #postprocess
            output_np = cv2.resize(output_np, (300, 1259))
            output_np_ori= output_np.copy()
            scale = np.max(seis_noise_ori - 0.5, axis=0)/np.max(output_np_ori - 0.5, axis=0)
            seis_noise_ori = seis_noise_ori - 0.5
            seis_noise_ori = seis_noise_ori/scale
            seis_noise_ori = seis_noise_ori + 0.5
            for j in range(300):
                thres = np.where(np.abs(diffnoise[:,j]) > threshold)[0][0]
                output_np[:thres, j] = seis_noise_ori[:thres, j]
            output_np = output_np - 0.5 + mean_noise 
            output_np = output_np * (noise_max - noise_min) + noise_min
            
            output_np = np.fliplr(output_np)
            pred_clean3[:,:,i] = output_np
            
        for i in tqdm(range(300)):
            seis_noise = pred_noise[:,i,:]
            seis_noise = np.fliplr(seis_noise)
            
            #preprocess
            noise_min = np.min(seis_noise)
            noise_max = np.max(seis_noise)
            seis_noise = (seis_noise - noise_min) / (noise_max - noise_min)
            mean_noise = np.mean(seis_noise)
            seis_noise = seis_noise - mean_noise + 0.5
            diffnoise = np.diff(seis_noise, axis=0)
            seis_noise_ori = seis_noise.copy()
            seis_noise = cv2.resize(seis_noise, (320, 1280))
            data = torch.tensor(seis_noise, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
            
            #prediction
            outputs = model(data)
            output_np = outputs[0, 0, :, :].cpu().numpy()
            
            #postprocess
            output_np = cv2.resize(output_np, (300, 1259))
            output_np_ori= output_np.copy()
            scale = np.max(seis_noise_ori - 0.5, axis=0)/np.max(output_np_ori - 0.5, axis=0)
            seis_noise_ori = seis_noise_ori - 0.5
            seis_noise_ori = seis_noise_ori/scale
            seis_noise_ori = seis_noise_ori + 0.5
            for j in range(300):
                thres = np.where(np.abs(diffnoise[:,j]) > threshold)[0][0]
                output_np[:thres, j] = seis_noise_ori[:thres, j]
            output_np = output_np - 0.5 + mean_noise 
            output_np = output_np * (noise_max - noise_min) + noise_min
            
            output_np = np.fliplr(output_np)
            pred_clean4[:,i,:] = output_np 
            
        pred_clean = (pred_clean + pred_clean2 + pred_clean3 + pred_clean4)/4  
        np.save(predictions_path+pred_files[i_file].split('/')[2].split('\\')[0]+'_gt.npy', 
                pred_clean.T)

Predicting 1 of 15


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:12<00:00, 23.37it/s]


Predicting 2 of 15


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:12<00:00, 24.16it/s]


Predicting 3 of 15


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:13<00:00, 22.59it/s]


Predicting 14 of 15


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:13<00:00, 22.25it/s]


Predicting 15 of 15


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:13<00:00, 22.09it/s]


### Submission File Generation

The ThinkOnward team has provided challengers with submission file generation code, which can be found in the `utils.py` script. There are two functions provided that will be useful; `create_single_submission()` and `create_submission()`.

`create_single_submission()` allows you to create a single submission file for any number of volumes. This should be used for testing purposes. Once you have predictions from your model, you can create a single submission file to test in the scoring function provided in the `Evaluation` section. 

`create_submission()` should be used as the final output submission file for challengers to provide to the scoring page.

Below is an example of how you might load some training data and create a submission file. While we encourage testing of the submission file generation and scoring code using training data, the final submission will be generated using the 15 test volumes.

For the sake of testing, create 3 randomly generated predictions using Numpy and save them to the predictions folder.

example1 = np.random.rand(1259, 300, 300)
example2 = np.random.rand(1259, 300, 300)
example3 = np.random.rand(1259, 300, 300)
np.save("./predictions/example1.npy", example1)
np.save("./predictions/example2.npy", example2)
np.save("./predictions/example3.npy", example3)

Next, you'll use these test predictions to create the `seismic_filenames` and `predictions` lists that contain the required info to feed into `create_submission()` below.

In [16]:
predictions_path = (
    predictions_path  # Path to your predictions folder or training data folder
)
seismic_filenames = sorted(
    [
        file.split("/")[0]
        for path, dirs, files in os.walk(predictions_path)
        for file in files
        if file.endswith(".npy")
    ]
)  # ensure that the filenames are sorted
print(seismic_filenames)

predictions = []
for seismic_filename in seismic_filenames:
    seismic = np.load(f"{predictions_path}{seismic_filename}")
    seismic = rescale_volume(seismic, low=0, high=100)#rescale, no clipping
    predictions.append(seismic)
    print(seismic_filename, "loaded")

['2024-06-10_0d6402b1_gt.npy', '2024-06-10_1a4e5680_gt.npy', '2024-06-10_1b9a0096_gt.npy', '2024-06-10_2bd82c05_gt.npy', '2024-06-10_3b118e17_gt.npy', '2024-06-10_43537d46_gt.npy', '2024-06-10_662066f4_gt.npy', '2024-06-10_971ac6dd_gt.npy', '2024-06-10_9871c8c6_gt.npy', '2024-06-10_b7c329be_gt.npy', '2024-06-10_bfd43f22_gt.npy', '2024-06-10_c952ed24_gt.npy', '2024-06-10_cec3da7f_gt.npy', '2024-06-10_eb45f27e_gt.npy', '2024-06-11_f46c20fe_gt.npy']
2024-06-10_0d6402b1_gt.npy loaded
2024-06-10_1a4e5680_gt.npy loaded
2024-06-10_1b9a0096_gt.npy loaded
2024-06-10_2bd82c05_gt.npy loaded
2024-06-10_3b118e17_gt.npy loaded
2024-06-10_43537d46_gt.npy loaded
2024-06-10_662066f4_gt.npy loaded
2024-06-10_971ac6dd_gt.npy loaded
2024-06-10_9871c8c6_gt.npy loaded
2024-06-10_b7c329be_gt.npy loaded
2024-06-10_bfd43f22_gt.npy loaded
2024-06-10_c952ed24_gt.npy loaded
2024-06-10_cec3da7f_gt.npy loaded
2024-06-10_eb45f27e_gt.npy loaded
2024-06-11_f46c20fe_gt.npy loaded


Let's say you want to create a single submission file just to test your model's quality. Here is code that creates one single submission file to feed into the scoring algorithm provided below. You can create your own ground truth file by using the denoised volumes provided to create a ground truth submission file for use in the scoring algorithm below. 

In [17]:
for seismic_filename, target in zip(seismic_filenames, predictions):
    create_single_submission(
        seismic_filename,
        prediction=target,
        submission_path="./submission_files/Final_Submission_Leo.npz",
    )

---

This notebook is licensed under the [MIT License](./LICENSE).